# Option Pricing Tool

In [ ]:
import sys
import os

# Add the directory where the module is located to the system path
sys.path.append(os.path.abspath(r'C:\Users\Devansh\Downloads\Python Projects\option-pricing-and-metrics-display-tool'))

from option_pricing import NeuralSDEModel, build_neural_network, train_neural_network
from option_pricing import BlackScholesModel, BinomialTreeModel, MonteCarloModel, HestonModel, MertonJumpDiffusionModel, HullWhiteModel, TrinomialTreeModel
from option_pricing import Visualizations, Greeks, VolatilityMeasures

import ipywidgets as widgets
from IPython.display import display, clear_output
import yfinance as yf
import numpy as np
import time
#from tensorflow.keras.models import load_model

In [ ]:
# Define UI elements for user inputs
ticker_input = widgets.Text(
    value='^NSEI',
    placeholder='Enter a ticker (e.g., ^NSEI)',
    description='Ticker:',
    disabled=False
)

strike_input = widgets.FloatText(
    value=23000,
    description='Strike Price:',
    disabled=False
)

maturity_input = widgets.FloatText(
    value=0.05,
    description='Time to Maturity (Years):',
    disabled=False
)

model_selection = widgets.Dropdown(
    options=['Black-Scholes', 'Binomial Tree', 'Monte Carlo', 'Heston', 'Merton Jump Diffusion', 'Hull-White', 'Trinomial'],
    value='Black-Scholes',
    description='Model:',
    disabled=False
)

chart_selection = widgets.Dropdown(
    options=['Price History', 'Volatility Comparison', 'Option Greeks'],
    value='Price History',
    description='Chart:',
    disabled=False
)

start_button = widgets.Button(
    description='Start Analysis',
    disabled=False,
    button_style='success',
    tooltip='Click to start the analysis',
    icon='check'
)

# Define labels to display results
price_label = widgets.Label(value="Option Price: ")
greeks_label = widgets.Label(value="Greeks: ")
volatility_label = widgets.Label(value="Volatility: ")

In [ ]:
def on_button_clicked(b):
    ticker = ticker_input.value
    strike_price = strike_input.value
    maturity = maturity_input.value
    selected_model = model_selection.value
    selected_chart = chart_selection.value

    # Clear output before starting the analysis
    clear_output(wait=True)
    
    # Display the UI components again to keep them visible during updates
    display(ticker_input, strike_input, maturity_input, model_selection, chart_selection, start_button, price_label, greeks_label, volatility_label)
    
    # Fetch live data
    data = yf.download(ticker, period='1d', interval='1m')
    S0 = data['Close'].iloc[-1]
    r = 0.065  # Example risk-free rate
    sigma = data['Close'].pct_change().std() * np.sqrt(252)  # Historical volatility

    # Initialize call_price to a default value
    call_price = None

    '''if selected_model == 'Neural-SDE':
        # Train a neural network on the historical data
        neural_net = build_neural_network()
        train_neural_network(data['Close'].values, neural_net, epochs=5)  # Reduced epochs for quicker training

        #Use the trained neural network in the Neural-SDE model
        #model = NeuralSDEModel(S0, strike_price, maturity, r, neural_net)
        #call_price = model.option_price(option_type='call')'''
    
    if selected_model == 'Black-Scholes':
        model = BlackScholesModel(S0=S0, X=strike_price, T=maturity, r=r, sigma=sigma)
        call_price = model.call_price()
    elif selected_model == 'Binomial Tree':
        model = BinomialTreeModel(S0=S0, X=strike_price, T=maturity, r=r, sigma=sigma, N=100)
        call_price = model.price()
    elif selected_model == 'Monte Carlo':
        model = MonteCarloModel(S0=S0, X=strike_price, T=maturity, r=r, sigma=sigma)
        call_price = model.option_price(option_type='call')
    elif selected_model == 'Heston':
        model = HestonModel(S0=S0, X=strike_price, T=maturity, r=r, kappa=2.0, theta=sigma**2, sigma=0.1, v0=sigma**2, rho=-0.7)
        call_price = model.option_price(option_type='call')
    elif selected_model == 'Merton Jump Diffusion':
        model = MertonJumpDiffusionModel(S0=S0, X=strike_price, T=maturity, r=r, sigma=sigma, lambda_=0.1, mu=-0.2, delta=0.1)
        call_price = model.option_price(option_type='call')
    elif selected_model == 'Hull-White':
        model = HullWhiteModel(S0=S0, X=strike_price, T=maturity, r0=r, sigma=sigma, kappa=0.03, theta=0.05, v0=0.01)
        call_price = model.option_price(option_type='call')
    elif selected_model == 'Trinomial':
        model = TrinomialTreeModel(S0=S0, X=strike_price, T=maturity, r=r, sigma=sigma, N=100)
        call_price = model.price()
    else:
        model = BlackScholesModel(S0=S0, X=strike_price, T=maturity, r=r, sigma=sigma)
        call_price = model.call_price()

    # Update the price label
    if call_price is not None:
        price_label.value = f"Option Price: {call_price:.2f}"
    else:
        price_label.value = "Option Price: N/A"

    # Calculate and update Greeks
    greeks = Greeks(S0=S0, X=strike_price, T=maturity, r=r, sigma=sigma)
    delta, gamma, theta, vega, rho = greeks.all_greeks()
    greeks_label.value = (f"Greeks: Delta={delta:.4f}, Gamma={gamma:.4f}, "
                         f"Theta={theta:.4f}, Vega={vega:.4f}, Rho={rho:.4f}")

    # Calculate and update Volatility Measures
    vol_measures = VolatilityMeasures(data['Close'])
    hist_vol = vol_measures.historical_volatility()
    imp_vol = sigma  # Placeholder for implied volatility
    volatility_label.value = f"Volatility: Historical={hist_vol:.4f}, Implied={imp_vol:.4f}"

    # Update Visualization based on user selection
    viz = Visualizations(data)
    if selected_chart == 'Price History':
        viz.plot_price_history(ticker)
    elif selected_chart == 'Volatility Comparison':
        viz.plot_volatility(hist_vol, imp_vol)
    elif selected_chart == 'Option Greeks':
        viz.plot_greeks(delta, gamma, theta, vega, rho)


In [ ]:
start_button.on_click(on_button_clicked)

## START THE TOOL BY RUNNING THIS LINE

In [ ]:
# Display the UI components
display(ticker_input, strike_input, maturity_input, model_selection, chart_selection, start_button, price_label, greeks_label, volatility_label)